In [53]:
options(repos = c(CRAN = "https://cloud.r-project.org"))

if (!require("pacman")) install.packages("pacman")
pacman::p_load(tidyverse, ggplot2, dplyr, lubridate, stringr, readxl, data.table, gdata, scales, data.table)

In [54]:
data.2010 <- read.csv('../data/data-2010.csv')
data.2011 <- read.csv('../data/data-2011.csv')
data.2012 <- read.csv('../data/data-2012.csv')
data.2013 <- read.csv('../data/data-2013.csv')
data.2014 <- read.csv('../data/data-2014.csv')
data.2015 <- read.csv('../data/data-2015.csv')

In [56]:
fix_type <- function(df) {
  df$org_parent <- as.character(df$org_parent)
  df
}

data.2011 <- fix_type(data.2011)

In [57]:
data.full <- bind_rows(data.2010, data.2011, data.2012, data.2013, data.2014, data.2015)